In [48]:
import os
from openai import OpenAI

# model_name = "gpt-4.1"
# base_url = "https://api.openai.com/v1"

model_name = "meta-llama/llama-4-scout-17b-16e-instruct"
base_url = "https://api.groq.com/openai/v1"

client = OpenAI(base_url=base_url, api_key="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")


system_prompt = """
You are a helpful assistant that can answer questions and help with tasks, such as drafting short research reports. Cite your sources when relevant. 

You have access to the following tools:

- search(query: str) -> str: Searches the web and returns summaries of top results.
- fetch(url: str) -> str: Fetches the content of a given URL and returns it as a markdown page.

You may call one tool per turn, for up to 2 turns, before giving your final answer.
In each turn, you should respond in the following format:

<think>
[your thoughts here]
</think>
<tool>
JSON with the following fields:
- name: The name of the tool to call
- args: A dictionary of arguments to pass to the tool (must be valid JSON)
</tool>

When you are done, give your final answer in the following format:

<answer>
[your final answer here]
</answer>
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the latest news in the NBA?"},
]

response = client.chat.completions.create(
    model=model_name,
    messages=messages, # type: ignore
)

response = response.choices[0].message.content # type: ignore
print(response)

<think>
To get the latest news in the NBA, I should perform a web search to find recent and reliable sources.
</think>
<tool>
{
  "name": "search",
  "args": {"query": "latest news in the NBA"}
}
</tool>


In [28]:
# Basic search tool

def search(query: str) -> str:
    """Searches the web and returns summaries of top results.
    
    Args:
        query: The search query string

    Returns:
        Formatted string with bullet points of top 10 results, each with title, source, url, and brief summary

    Examples:
        {"query": "who invented the lightbulb"} -> ["Thomas Edison (1847-1931) - Inventor of the lightbulb", ...]
        {"query": "what is the capital of France"} -> ["Paris is the capital of France", ...]
        {"query": "when was the Declaration of Independence signed"} -> ["The Declaration of Independence was signed on July 4, 1776", ...]
    """

    try:
        from brave import Brave
        # set BRAVE_API_KEY in your environment
        brave = Brave(api_key="BSAuwioJ2usGIWaB4HggCKD-2_oZgt0")
        results = brave.search(q=query, count=10, raw=True) # type: ignore

        web_results = results.get('web', {}).get('results', []) # type: ignore

                
        if not web_results:
            return "No results found"
        
        summaries = []
        for r in web_results:
            if 'profile' not in r:
                continue
            header = f"{r['profile']['name']} ({r['profile']['long_name']})"
            title = r['title']
            snippet = r['description']
            url = r['url'] 
            summaries.append(f"•  {header}\n   {title}\n   {snippet}\n   {url}")

        return "\n\n".join(summaries)
        
    except Exception as e:
        return f"Error: {str(e)}"
    
results = search("latest news in the NBA")

print(results)

•  NBA (nba.com)
   NBA News - Latest team, player and league news | NBA.com
   <strong>NBA</strong> <strong>News</strong>: Your source of the most updated official <strong>NBA</strong> <strong>news</strong>. Stay current on the league, team and player <strong>news</strong>, scores, stats, standings from <strong>NBA</strong>.
   https://www.nba.com/news

•  NBA (nba.com)
   Top Stories | NBA.com
   Indy tries to punch its ticket to the Finals tonight, while New York seeks a Game 7 back at The Garden.
   https://www.nba.com/news/category/top-stories

•  ESPN (espn.co.uk)
   NBA on ESPN - Scores, Stats and Highlights
   Here&#x27;s everything you need to know about the 2025 <strong>NBA</strong> playoffs, including scores, <strong>news</strong> and highlights.
   https://www.espn.co.uk/nba/

•  BBC (British Broadcasting Corporation)
   NBA News, Results & Scores - Basketball - BBC Sport
   The home of Basketball on BBC Sport online. Includes <strong>the</strong> <strong>latest</strong> <s

In [11]:
# fetch tool for URL contents
def fetch(url: str) -> str:
    """Fetches the content of a given URL and returns it as a markdown page.
    
    Args:
        url: The URL to fetch the content from

    Returns:
        A markdown page with the content of the URL.
    """
    import requests
    from markdownify import markdownify
    
    response = requests.get(url)
    response.raise_for_status()
    return markdownify(response.text)
    
content = fetch("https://www.flashscoreusa.com/basketball/")
print(content)

Basketball Livescore, Basketball Results | Flashscore - NBA, Euroleague, NCAA




Basketball Livescore, Basketball Results, NBA, Euroleague, NCAA



![](https://static.flashscore.com/res/_fs/image/2_others/bg.png)

[Scores](/)
[News](/news/)




[Favorites](/favorites/)

[Soccer](/)
[Tennis](/tennis/)
[Golf](/golf/)
[Football](/football/)
[Basketball](/basketball/)
[Baseball](/baseball/)
[Hockey](/hockey/)

[Aussie rules](/aussie-rules/)
[Badminton](/badminton/)
[Baseball](/baseball/)
[Basketball](/basketball/)
[Beach volleyball](/beach-volleyball/)
[Boxing](/boxing/)
[Cricket](/cricket/)
[Cycling](/cycling/)
[Darts](/darts/)
[eSports](/esports/)
[Field hockey](/field-hockey/)
[Football](/football/)
[Futsal](/futsal/)
[Golf](/golf/)
[Hockey](/hockey/)
[Horse racing](/horse-racing/)
[MMA](/mma/)
[Motorsport](/motorsport/)
[Rugby League](/rugby-league/)
[Rugby Union](/rugby-union/)
[Snooker](/snooker/)
[Soccer](/soccer/)
[Table tennis](/table-tennis/)
[Team handball](/team-handball/)
[Te

In [26]:
import re
import json

def parse_thinking_from_response(response: str) -> str | None:
    """Parse a thinking from a response."""
    # re.DOTALL is used to all \n inside the think tags
    # ? matches lazily meaning we pick the content inside the first <think> </think>
    thinking = re.search(r'<think>(.*?)</think>', response, re.DOTALL)
    if thinking:
        return thinking.group(1)
    return None

def parse_tool_from_response(response: str) -> dict | None:
    """Parse a tool from a response."""
    tool_call = re.search(r'<tool>(.*?)</tool>', response, re.DOTALL)
    if tool_call:
        return json.loads(tool_call.group(1))
    return None


def parse_answer_from_response(response: str) -> str | None:
    """Parse an answer from a response."""
    answer = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
    if answer:
        return answer.group(1)
    return None

def call_tool(tool_call: dict) -> str:
    """Call a tool with the given tool call."""
    if tool_call['name'] == 'search':
        return search(tool_call['args']['query'])
    elif tool_call['name'] == 'fetch':
        return fetch(tool_call['args']['url'])
    else:
        return f"Error: Tool {tool_call['name']} not found"

example_response = """
<think>
To get the latest news in the NBA, I should perform a web search to find recent and reliable sources.
</think>
<tool>
{
  "name": "search",
  "args": {"query": "latest news in the NBA"}
}
</tool>
"""

tool_call = parse_tool_from_response(example_response) # type: ignore
print(tool_call)
call_tool(tool_call)

{'name': 'search', 'args': {'query': 'latest news in the NBA'}}


'•  NBA (nba.com)\n   NBA News - Latest team, player and league news | NBA.com\n   <strong>NBA</strong> <strong>News</strong>: Your source of the most updated official <strong>NBA</strong> <strong>news</strong>. Stay current on the league, team and player <strong>news</strong>, scores, stats, standings from <strong>NBA</strong>.\n   https://www.nba.com/news\n\n•  NBA (nba.com)\n   Top Stories | NBA.com\n   Indy tries to punch its ticket to the Finals tonight, while New York seeks a Game 7 back at The Garden.\n   https://www.nba.com/news/category/top-stories\n\n•  ESPN (espn.co.uk)\n   NBA on ESPN - Scores, Stats and Highlights\n   Here&#x27;s everything you need to know about the 2025 <strong>NBA</strong> playoffs, including scores, <strong>news</strong> and highlights.\n   https://www.espn.co.uk/nba/\n\n•  BBC (British Broadcasting Corporation)\n   NBA News, Results & Scores - Basketball - BBC Sport\n   The home of Basketball on BBC Sport online. Includes <strong>the</strong> <strong>

In [41]:
from time import sleep

final_answer = ""

question = "What is the latest news in the NBA? Give me a 3 paragraph report."
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question},
]

turns = 0
while True:
    turns += 1
    retries = 0 
    while retries < 5:
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=messages, # type: ignore
            )
            response = response.choices[0].message.content # type: ignore
            # parse for thinking, tool call, and/or answer 
            maybe_thinking = parse_thinking_from_response(response) # type: ignore
            maybe_tool_call = parse_tool_from_response(response) # type: ignore
            maybe_answer = parse_answer_from_response(response) # type: ignore
            if maybe_tool_call or maybe_answer:
                break
        except Exception as e:
            print(f"Error: {e}")
            retries += 1

    print("=== Turn", turns, "===")
    
    if maybe_thinking: # type: ignore

        thinking = maybe_thinking.strip()
        print(f"Thinking: {thinking}")

    if maybe_tool_call: # type: ignore
        tool_call = maybe_tool_call
        tool_result = call_tool(tool_call)

        sleep(2)
        
        print(f"Tool call: {tool_call}")
        print(f"Tool result: {tool_result[:100]}")

        messages.append({"role": "user", "content": tool_result})

    elif maybe_answer: # type: ignore
        final_answer = maybe_answer
        break

    else:
        print("Error: No tool call or answer found")
        break

    if turns == 4:
        break

=== Turn 1 ===
Thinking: To provide a 3-paragraph report on the latest news in the NBA, I will first search for current NBA news. This will help me gather the most recent and relevant information.
Tool call: {'name': 'search', 'args': {'query': 'latest news in the NBA'}}
Tool result: •  NBA (nba.com)
   NBA News - Latest team, player and league news | NBA.com
   <strong>NBA</strong>
=== Turn 2 ===
Thinking: To provide a 3-paragraph report on the latest news in the NBA, I will need to visit some of these websites to gather current information. I'll start by visiting the official NBA website to get the most updated news.
Tool call: {'name': 'fetch', 'args': {'url': 'https://www.nba.com/news'}}
Tool result: NBA News - Latest team, player and league news | NBA.com

Navigation Toggle[![NBA Logo](https://cdn.
=== Turn 3 ===
Thinking: I will start by visiting the official NBA website to get the latest news and updates.
Tool call: {'name': 'fetch', 'args': {'url': 'https://www.nba.com/news'}}


In [42]:
print(f"Final answer: {final_answer}")

Final answer: 


In [45]:
### Models as tools

def ask_model(question: str, url: str) -> str:
    """Ask a model a question about a URL and return the answer."""

    fetch_result = fetch(url)
    system_prompt = "You are a helpful assistant that can answer questions about a given URL."
    prompt = f"""
    Here is the content of the URL {url}:
    {fetch_result}

    Here is the question:
    {question}
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    # ask model to answer question about url
    answer = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages, # type: ignore
    )
    return answer.choices[0].message.content # type: ignore

# test
question = "What is the latest news in the NBA?."
url = "https://www.nba.com/news"
answer = ask_model(question, url)
print(answer)

The latest news in the NBA includes:

1. The Thunder's week-long gap between games and how it may impact their chances in the NBA Finals.
2. 50 early entry candidates withdrawing from the NBA Draft 2025.
3. The Oklahoma City Thunder advancing to the 2025 NBA Finals.
4. The Eastern Conference Finals between the Knicks and Pacers, with the series currently at 3-2 in favor of the Pacers.
5. The Timberwolves' gap between them and a championship, despite their progress since drafting Anthony Edwards.
6. Shai Gilgeous-Alexander being named the 2024-25 Kia NBA Most Valuable Player.
7. The Knicks staving off elimination to force Game 6 against the Pacers.

These are just a few of the latest news stories in the NBA, and there are many more articles and updates available on the NBA website.


In [46]:
import os
from openai import OpenAI

model_name = "meta-llama/llama-4-scout-17b-16e-instruct"
base_url = "https://api.groq.com/openai/v1"

client = OpenAI(base_url=base_url, api_key="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")

system_prompt = """
You are a helpful assistant that can answer questions and help with tasks, such as drafting short research reports. Cite your sources when relevant. 

You have access to the following tools:

- search(query: str) -> str: Searches the web and returns summaries of top results.
- ask_model(question: str, url: str) -> str: Ask an AI helper model a question about a given URL and return the answer.

You may call one tool per turn, for up to 10 turns, before giving your final answer.


In each turn, you should respond in the following format:

<think>
[your thoughts here]
</think>
<tool>
JSON with the following fields:
- name: The name of the tool to call
- args: A dictionary of arguments to pass to the tool (must be valid JSON)
</tool>

When you are done, give your final answer in the following format:

<answer>
[your final answer here]
</answer>
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the latest news in the NBA?"},
]

response = client.chat.completions.create(
    model=model_name,
    messages=messages, # type: ignore
)

response = response.choices[0].message.content # type: ignore
print(response)

<think>
To find the latest news in the NBA, I should perform a web search to get the most recent updates.
</think>
<tool>
{
  "name": "search",
  "args": {"query": "latest news in the NBA"}
}
</tool>


In [47]:
# updated call_tool
def call_tool(tool_call: dict) -> str:
    """Call a tool with the given tool call."""
    if tool_call['name'] == 'search':
        return search(tool_call['args']['query'])
    elif tool_call['name'] == 'fetch':
        return fetch(tool_call['args']['url'])
    elif tool_call['name'] == 'ask_model':
        return ask_model(tool_call['args']['question'], tool_call['args']['url'])
    else:
        return f"Error: Tool {tool_call['name']} not found"
    
# agent loop
final_answer = ""

question = "What is the latest news in the NBA? Give me a 3 paragraph report."
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question},
]

turns = 0
while True:
    turns += 1
    retries = 0 
    while retries < 5:
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=messages, # type: ignore
            )
            response = response.choices[0].message.content # type: ignore
            # parse for thinking, tool call, and/or answer 
            maybe_thinking = parse_thinking_from_response(response) # type: ignore
            maybe_tool_call = parse_tool_from_response(response) # type: ignore
            maybe_answer = parse_answer_from_response(response) # type: ignore
            if maybe_thinking or maybe_tool_call or maybe_answer:
                break
        except Exception as e:
            print(f"Error: {e}")
            retries += 1
    print("=== Turn", turns, "===")
    if maybe_thinking: # type: ignore
        thinking = maybe_thinking.strip()
        print(f"Thinking: {thinking}")
    if maybe_tool_call: # type: ignore
        tool_call = maybe_tool_call
        tool_result = call_tool(tool_call)
        print(f"Tool call: {tool_call}")
        print(f"Tool result: {tool_result[:100]}")
        messages.append({"role": "user", "content": tool_result})
    elif maybe_answer: # type: ignore
        final_answer = maybe_answer
        break
    else:
        print("Error: No tool call or answer found")
        break

=== Turn 1 ===
Thinking: To provide a 3-paragraph report on the latest news in the NBA, I will first need to search for current and relevant information. I will use the search tool to find the latest news.
Tool call: {'name': 'search', 'args': {'query': 'latest news in the NBA'}}
Tool result: •  NBA (nba.com)
   NBA News - Latest team, player and league news | NBA.com
   <strong>NBA</strong>
=== Turn 2 ===
Thinking: To provide a 3-paragraph report on the latest news in the NBA, I will need to visit some of these websites to gather current information. I'll start with the official NBA website (nba.com) to get the most up-to-date news.
Tool call: {'name': 'visit', 'args': {'url': 'https://www.nba.com/news'}}
Tool result: Error: Tool visit not found
=== Turn 3 ===
Thinking: I need to search for the latest NBA news and summarize it into a 3-paragraph report. I'll start by using the search tool to find recent news articles about the NBA.
Tool call: {'name': 'search', 'args': {'query': 'late

KeyboardInterrupt: 